In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
df = pd.read_csv('C:\\Users\\Hardik\\Desktop\\ML_Pipeline\\02-logistic-regression-heart\\heart_cleveland_upload.csv')  # download from Kaggle first
df.head()


In [ ]:
print("Missing values:\n", df.isnull().sum())

In [ ]:
print("\nData types:\n", df.dtypes)

In [ ]:
categorical_features = ['cp', 'thal', 'slope']
print("\nCategorical features:\n", categorical_features)

In [ ]:
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

In [ ]:
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
scaler = StandardScaler()
df_encoded[numerical_features] = scaler.fit_transform(df_encoded[numerical_features])

In [ ]:
print("\nProcessed data shape:", df_encoded.shape)
df_encoded.head()

In [ ]:
print(df.columns.tolist())



In [ ]:
print(df.columns.tolist())


In [ ]:
X = df.drop('condition', axis=1)
y = df['condition']


In [ ]:
print(y.value_counts())


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)


In [ ]:
y_pred = model.predict(X_test_scaled)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Accuracy
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Visualize and save
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")

# Save the image
plt.savefig("confusion_matrix.png", dpi=300, bbox_inches='tight')  # Save with high resolution

plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Get predicted probabilities for the positive class
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# Calculate FPR, TPR
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# Plot ROC Curve
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label='Logistic Regression')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal (random chance)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()

# Save the image
plt.savefig("roc_curve.png", dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

# Print AUC Score
print("AUC Score:", roc_auc_score(y_test, y_prob))
